# Data Querying

You can execute queries and get back the result as a pandas DataFrame.

There are two ways to query data:

- with cube.query by passing a list of measure and level objects
- with session.query_mdx by passing an MDX string

## Querying current session

In [ ]:
import atoti as tt

session = tt.create_session()
store = session.read_csv("data/sales.csv", keys=["Sale ID"])
cube = session.create_cube(store, "Sales")

### Querying a cube

In [ ]:
m = cube.measures
lvl = cube.levels
cube.query(m["Amount.SUM"], m["Quantity.SUM"], levels=[lvl["Product"]])

### MDX Query

In [ ]:
mdx_query = """
SELECT
  NON EMPTY [Measures].[Amount.SUM] ON COLUMNS,
  NON EMPTY [Hierarchies].[Product].[Product].Members ON ROWS
  FROM (
    SELECT
    {
      [Hierarchies].[Date].[ALL].[AllMember].[2020-05-14],
      [Hierarchies].[Date].[ALL].[AllMember].[2020-05-15],
      [Hierarchies].[Date].[ALL].[AllMember].[2020-05-16]
    } ON COLUMNS
    FROM [Sales]
  )
"""

In [ ]:
session.query_mdx(mdx_query)

## Querying remote cube

It is possible to connect to an existing cube and query it.
This cube can be started by another atoti process or even a classic ActivePivot (version >=5.7).

To do that you need to login and open a query session that will only let you query the cubes but not modify them.

In this example we will connect to the session created before with atoti:

In [ ]:
existing_session = tt.open_query_session(
    session.url,  # Here you would put the base URL of the existing session.
    # if we needed authentication we could pass it like that
    # auth=tt.query.create_basic_authentication("admin", "admin"),
)
existing_session

### Querying a remote cube

You can retrieve the cubes of the remote session to access their levels and measures and query it.

In [ ]:
existing_cube = existing_session.cubes["Sales"]
lvl = existing_cube.levels
m = existing_cube.measures
existing_cube.query(m["Amount.SUM"], m["Quantity.SUM"], levels=[lvl["Product"]])

### Remote MDX query

In [ ]:
existing_session.query_mdx(mdx_query)